In [1]:
import csv
import chart_studio
import re
import string
# import emot
import collections
import ipywidgets
import contractions
import cufflinks
import nltk.tokenize

import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np

# from textblob import TextBlob
#from google.colab import widgets
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

pd.set_option('display.max_colwidth', None)
sid = SentimentIntensityAnalyzer()
# emot_obj = emot.core.emot()
cufflinks.set_config_file(world_readable=True, theme='pearl')
cufflinks.go_offline()
pio.renderers.default = 'colab'

In [2]:
# Reading the CSV to DF & showing DF info
df = pd.read_csv('old_town_road.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              1265 non-null   object 
 1   vader_neg       1264 non-null   float64
 2   vader_neu       1264 non-null   float64
 3   vader_pos       1264 non-null   float64
 4   vader_compound  1264 non-null   float64
 5   roberta_neg     1264 non-null   float64
 6   roberta_neu     1264 non-null   float64
 7   roberta_pos     1264 non-null   float64
 8   Text            1264 non-null   object 
dtypes: float64(7), object(2)
memory usage: 89.1+ KB


In [3]:
# Drop duplicates from text column
df = df.drop_duplicates(subset = ['Text'],keep='first')
print(df)

        Id  vader_neg  vader_neu  vader_pos  vader_compound  roberta_neg  \
0        1      0.000      1.000      0.000          0.0000     0.164698   
1        2      0.000      0.000      1.000          0.4404     0.060793   
2        3      0.000      1.000      0.000          0.0000     0.400678   
3        4      0.000      1.000      0.000          0.0000     0.011754   
4        5      0.000      0.911      0.089          0.4824     0.090882   
...    ...        ...        ...        ...             ...          ...   
1260  1263      0.000      1.000      0.000          0.0000     0.114758   
1261  1264      0.000      0.000      0.000          0.0000     0.259181   
1262  1265      0.000      1.000      0.000          0.0000     0.230317   
1263  1266      0.000      1.000      0.000          0.0000     0.110747   
1264  1267      0.403      0.597      0.000         -0.8442     0.870241   

      roberta_neu  roberta_pos  \
0        0.707790     0.127512   
1        0.329428  

In [4]:
def pre_process(text):
    # Remove links
    text = re.sub('http://\S+|https://\S+', '', str(text))
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)

    text = re.sub('&amp', 'and', text)
    text = re.sub('&lt', '<', text)
    text = re.sub('&gt', '>', text)

    # Remove new line characters
    text = re.sub('[\r\n]+', ' ', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)

    # Remove multiple space characters
    text = re.sub('\s+',' ', text)
    
    return text

In [5]:
# Apply preprocessing to the dataframe / define the column with the comment text
df['processed_text'] = df['Text'].apply(pre_process)

In [6]:
df.sample(n=5)

,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,Text,processed_text
1074,1077,0.000,0.286,0.714,0.3612,0.132917,0.636686,0.230398,like me,like me
498,501,0.078,0.922,0.000,-0.1557,0.427299,0.525711,0.046991,"He was gay since he has 16, he confirmed it on his twitter but he was less weird than now","He was gay since he has 16, he confirmed it on his twitter but he was less weird than now"
224,225,0.000,1.000,0.000,0.0000,0.208443,0.737671,0.053886,"<a href=""https://www.youtube.com/watch?v=r7qovpFAGrQ&amp;t=1m39s"">1:39</a>","<a href="""
1096,1099,0.000,0.278,0.722,0.3818,0.083052,0.783314,0.133634,@Octopi Ty,Ty
1172,1175,0.000,1.000,0.000,0.0000,0.535320,0.319090,0.145590,Bro what💀,Bro what💀


In [7]:
# Dropping rows with href URL
df = df[df["Text"].str.contains(r'href=\S+') == False]
df = df[df["Text"].str.contains(r'https:\S+') == False]
print(df)

        Id  vader_neg  vader_neu  vader_pos  vader_compound  roberta_neg  \
0        1      0.000      1.000      0.000          0.0000     0.164698   
1        2      0.000      0.000      1.000          0.4404     0.060793   
2        3      0.000      1.000      0.000          0.0000     0.400678   
3        4      0.000      1.000      0.000          0.0000     0.011754   
4        5      0.000      0.911      0.089          0.4824     0.090882   
...    ...        ...        ...        ...             ...          ...   
1260  1263      0.000      1.000      0.000          0.0000     0.114758   
1261  1264      0.000      0.000      0.000          0.0000     0.259181   
1262  1265      0.000      1.000      0.000          0.0000     0.230317   
1263  1266      0.000      1.000      0.000          0.0000     0.110747   
1264  1267      0.403      0.597      0.000         -0.8442     0.870241   

      roberta_neu  roberta_pos  \
0        0.707790     0.127512   
1        0.329428  

In [8]:
# Dropping rows with non English words
df=df[df.Text.map(lambda x: x.isascii())]
print(df)

        Id  vader_neg  vader_neu  vader_pos  vader_compound  roberta_neg  \
0        1      0.000      1.000      0.000          0.0000     0.164698   
1        2      0.000      0.000      1.000          0.4404     0.060793   
5        6      0.000      1.000      0.000          0.0000     0.258892   
7        8      0.272      0.435      0.293          0.0516     0.975138   
8        9      0.000      0.000      1.000          0.4215     0.115661   
...    ...        ...        ...        ...             ...          ...   
1259  1262      0.000      0.000      0.000          0.0000     0.285115   
1260  1263      0.000      1.000      0.000          0.0000     0.114758   
1261  1264      0.000      0.000      0.000          0.0000     0.259181   
1262  1265      0.000      1.000      0.000          0.0000     0.230317   
1263  1266      0.000      1.000      0.000          0.0000     0.110747   

      roberta_neu  roberta_pos  \
0        0.707790     0.127512   
1        0.329428  

In [9]:
#Remove or replacing API symbols with originals each one by one
# df = df.replace('&#39;', " ' ", regex = True)
# df = df.replace('&quot;', " ' ", regex = True)
# df = df.replace('&amp;', ' & ', regex = True)
# df = df.replace('&lt;', " < ", regex = True)
# df = df.replace('<b>', ' ', regex = True)
# df = df.replace('<br>', ' ', regex = True)
# df = df.replace('</b>', ' ', regex = True)
# df = df.replace('@', ' ', regex = True)
# print(df)

In [10]:
# Remove or replacing API symbols with originals chained
# df.Text = df.Text.str.replace("&#39;", " ' ")\
# .str.replace("&amp;", " & ")\
# .str.replace("&lt;", " < ")\
# .str.replace("&quot;", " ' ")\
# .str.replace("<b>", " ")\
# .str.replace("<br>", " ")\
# .str.replace("</b>", " ")\
# .str.replace("@", " ")


In [11]:
df.sample(n=5)

,Id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,Text,processed_text
333,334,0.0,0.594,0.406,0.6249,0.008359,0.169581,0.822060,Only a masterpiece because of Nine Inch Nails.,Only a masterpiece because of Nine Inch Nails.
1054,1057,0.0,0.698,0.302,0.3818,0.014791,0.362505,0.622704,@Coach Ty <br>Yea that sounds about right.,Ty <br>Yea that sounds about right.
1198,1201,0.0,0.811,0.189,0.5994,0.059501,0.609329,0.331170,"Hat down, cross-town, livin&#39; like a rockstar\r<br>Spent a lot of money on my brand-new guitar\r<br>Baby&#39;s got a habit: diamond rings and Fendi sports bras","Hat down, cross-town, livin&; like a rockstar <br>Spent a lot of money on my brand-new guitar <br>Baby&;s got a habit: diamond rings and Fendi sports bras"
998,1001,0.0,0.278,0.722,0.6369,0.002389,0.097938,0.899673,"@WilliamAnthonyLeasure777 Yes, Thank you.","Yes, Thank you."
187,188,0.0,1.000,0.000,0.0000,0.204512,0.552149,0.243340,hi,hi


In [12]:
# Display the 20 most present bi-grams
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_bigram(df['processed_text'], 20)


In [13]:
df1 = pd.DataFrame(common_words, columns = ['Comment_Text' , 'count'])
df1.groupby('Comment_Text').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar',
    yTitle='Count',
    linecolor='black',
    title='Top 20 bigrams in Comment before removing spams')

# Building a Spam Detection Model

In [14]:
# Import data
df_spam1 = pd.read_csv("yt_spam_data/Youtube01-Channel1.csv")
df_spam2 = pd.read_csv("yt_spam_data/Youtube02-Channel2.csv")
df_spam3 = pd.read_csv("yt_spam_data/Youtube03-Channel3.csv")
df_spam4 = pd.read_csv("yt_spam_data/Youtube04-Channel4.csv")
df_spam5 = pd.read_csv("yt_spam_data/Youtube05-Channel5.csv")
df_spam6 = pd.read_csv("yt_spam_data/Youtube06-Channel6.csv")
# df_spam6 = pd.read_csv("yt_spam_data/5000_yt_comments.csv", encoding= 'unicode_escape')

In [15]:
# Merge data
frames = [df_spam1,df_spam2,df_spam3,df_spam4,df_spam5,df_spam6]
df_merged = pd.concat(frames)
df_merged

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: kobyoshi02",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,"Hey guys check out my new channel and our first vid THIS IS US THE MONKEYS!!! I'm the monkey in the white shirt,please leave a like comment and please subscribe!!!!",1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
204,205,latrell brown,2013-10-08T18:26:06.234000,Driver side same as the Passenger you just manually went lightly on the drivers side.,0
205,206,OnlineJobs99,2013-10-07T03:35:34.267000,"Now, put your finger there.",0
206,207,WhatUKnow,2013-10-07T02:59:17.497000,What? No hand test on the passenger side?,0
207,208,jameskileymusic,2013-10-06T05:38:47.524000,"go on then, put that finger in there",0


In [16]:
# Assign keys to each dataset
keys = ["Channel1","Channel2","Channel3","Channel4","Channel5","Channel6"]


In [17]:
# Concat keys
df_with_keys = pd.concat(frames,keys=keys)
df_with_keys

COMMENT_ID            AUTHOR  \
Channel1 0    LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU         Julius NM   
         1    LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A       adam riyati   
         2    LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8  Evgeny Murashkin   
         3            z13jhp0bxqncu512g22wvzkasxmvvzjaz04   ElNino Melendez   
         4            z13fwbwp1oujthgqj04chlngpvzmtt3r3dw            GsMega   
...                                                   ...               ...   
Channel6 204                                          205     latrell brown   
         205                                          206      OnlineJobs99   
         206                                          207         WhatUKnow   
         207                                          208   jameskileymusic   
         208                                          209         WhatUKnow   

                                    DATE  \
Channel1 0           2013-11-07T06:20:48   
         1           2013-11-07T12:37:15   
         2           2013-11-08T17:34:21   
         3           2013-11-09T08:28:43   
         4           2013-11-10T16:05:38   
...                                  ...   
Channel6 204  2013-10-08T18:26:06.234000   
         205  2013-10-07T03:35:34.267000   
         206  2013-10-07T02:59:17.497000   
         207  2013-10-06T05:38:47.524000   
         208  2013-10-06T03:24:16.073000   

                                                                                                                                                                             CONTENT  \
Channel1 0                                                                                                                  Huh, anyway check out this you[tube] channel: kobyoshi02   
         1    Hey guys check out my new channel and our first vid THIS IS US THE  MONKEYS!!! I'm the monkey in the white shirt,please leave a like comment  and please subscribe!!!!   
         2                                                                                                                                    just for test I have to say murdev.com   
         3                                                                                                                          me shaking my sexy ass on my channel enjoy ^_^ ﻿   
         4                                                                                                                                   watch?v=vtaRGgvGtWQ   Check this out .﻿   
...                                                                                                                                                                              ...   
Channel6 204                                                                                   Driver side same as the Passenger you just manually went lightly on the drivers side.   
         205                                                                                                                                             Now, put your finger there.   
         206                                                                                                                               What? No hand test on the passenger side?   
         207                                                                                                                                    go on then, put that finger in there   
         208                                                                                              bro saw the carrot getting cut on the passenger side and was like heck nah   

              CLASS  
Channel1 0        1  
         1        1  
         2        1  
         3        1  
         4        1  
...             ...  
Channel6 204      0  
         205      0  
         206      0  
         207      0  
         208      0  

[2163 rows x 5 columns]

In [18]:
# Extract the important columns
df_data = df_with_keys[["CONTENT","CLASS"]]

In [19]:
# Specify input (x) and output (y) variables
df_x = df_data['CONTENT']
df_y = df_data['CLASS']

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# Feature extraction from input variable
corpus = df_x
cv = CountVectorizer()
X = cv.fit_transform(corpus)

In [22]:
# Convert the numeric values into an array
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [24]:
# Split 70% of data for training the model, 30% to test the model
X_train, X_test, y_train, y_test = train_test_split(X, df_y, test_size=0.30, random_state=42)

In [25]:
# Initialize MultinomialNB
clf = MultinomialNB()

In [26]:
# Fit the model
clf.fit(X_train,y_train)

MultinomialNB()

In [27]:
# Calculate the accuracy score of the model
print("Accuracy of Model",clf.score(X_test,y_test)*100,"%")

Accuracy of Model 91.52542372881356 %


In [28]:
# Classify YT comments
clf.predict(X_test)

array([0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,

In [29]:
# Test the model - spam(1) or non-spam(0)
comment = ["i think you will like my new song"]
vect = cv.transform(comment).toarray()
clf.predict(vect)

array([1])

In [30]:
comment1 = ["baited gomunkul 6 style"]
vect = cv.transform(comment1).toarray()
clf.predict(vect)

array([0])

In [31]:
def expand_contractions(text):
  try:
    return contractions.fix(text)
  except:
    return text

In [32]:
not_df = pd.read_json("all_channels_comments.json")
comment_list = not_df["comments"].tolist()

In [33]:
count = 0
for x in range(0,len(comment_list)):
    count += len(comment_list[x])

count

253409

In [34]:
for x in range(0,len(not_df)):
    try:
        temp_list = comment_list[x]
        temp_df = pd.DataFrame(temp_list,columns=["Text"])
        temp_comments = (temp_df["Text"])
        temp_vect = cv.transform(temp_comments).toarray()
        temp_pred = clf.predict(temp_vect)
        temp_vect_list = temp_pred.tolist()
        temp_df_vect = pd.DataFrame({"results":temp_vect_list})
        inner_joined = pd.concat([temp_df,temp_df_vect],axis="columns",join="inner")
        spam_dropped = inner_joined[inner_joined.results != 1]
        spam_dropped['expanded_text'] = spam_dropped['Text'].apply(expand_contractions)
        spam_dropped['processed_text'] = spam_dropped['expanded_text'].apply(pre_process)
        comment_list[x] = spam_dropped["processed_text"].tolist()
    except Exception as e:
        print(e)
        pass


In [35]:
count = 0
for x in range(0,len(comment_list)):
    count += len(comment_list[x])

count

189762

In [36]:
not_df.to_json("all_comments_clean.json")
check_df = pd.read_json("all_comments_clean.json")

category                                   channel  \
0  Automobiles  https://www.youtube.com/@autofanaticcars   
1  Automobiles  https://www.youtube.com/@autofanaticcars   
2  Automobiles  https://www.youtube.com/@autofanaticcars   
3  Automobiles      https://www.youtube.com/@AutoAmateur   
4  Automobiles      https://www.youtube.com/@AutoAmateur   

                                     video_url     video_id  \
0  https://www.youtube.com/watch?v=Wqcy9do6YO0  Wqcy9do6YO0   
1  https://www.youtube.com/watch?v=NyNF0MfQN8s  NyNF0MfQN8s   
2  https://www.youtube.com/watch?v=x3vzurNYbyM  x3vzurNYbyM   
3  https://www.youtube.com/watch?v=qysq4YFOwPw  qysq4YFOwPw   
4  https://www.youtube.com/watch?v=B8zgCdINaAs  B8zgCdINaAs   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [38]:
check_df["comments"][0]

['Wheel Cleaner is In production and pre-order is officially open please visit auto-fanatic.com',
 '8 months? Is that all.',
 'impressed very effective',
 'No matter what, you have to use brushes and agitate ......',
 'Good video BUT, why don\'t you "articulate" that wand from a "below angle" in order to hit the spokes at 11 to 2 O\'clock which remained virtually un-touched by both the pre-solution AND the foam???  Drives me a bit crazy to see that, LOL!',
 'My wheels werenever done until 215k km, they looked better than those here',
 'Your finger bumps took the dust off lol',
 'Hope to see a cleaner eventually that can be used without a foam machine. Not buying a foam archive just want to be able to soeay my wheels with the cleaner from a spray bottle type. To expensive to buy a foamer and the solution.',
 "I've had my second hand car for eight years and only cleaned it now for the first time the wheel barrels by using oven cleaner and drill brush to get the same results you did.",
 '

In [37]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 317 entries, 0 to 316
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      317 non-null    object
 1   channel       317 non-null    object
 2   video_url     317 non-null    object
 3   video_id      317 non-null    object
 4   comments      317 non-null    object
 5   channel_size  317 non-null    int64 
 6   identity      317 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 19.8+ KB


In [35]:
# to_drop = ["i think you will like my new song", "Repent to Allah in Islam because","HOLY BIBLE", "school ganstas on Gomunkul 6", "youth on Gomunkul 6", \
#    "TRYNA SOUND LIKE GOMUNKUL 6", "normal people listening Gomunkul 6", "baited gomunkul 6 style", "uploaded a song to my channel if you want", "MONEYYY"]


In [36]:
# df = df[~df['Text'].str.contains('|'.join(to_drop))]

In [39]:
# spam_dropped.sample(n=5)

,Text,results,expanded_text,processed_text
149,this is what happens when we make marijuana illegal,0,this is what happens when we make marijuana illegal,this is what happens when we make marijuana illegal
152,"they died from methadone, the papers are full of shit looooool",0,"they died from methadone, the papers are full of shit looooool","they died from methadone, the papers are full of shit looooool"
17,I hate people that abuse drugs and give them a bad name. Bc I don't feel anything unless I take a stimulant or depressant like opioids bc I'm in pain all the time. Mentally and physically.,0,I hate people that abuse drugs and give them a bad name. Because I do not feel anything unless I take a stimulant or depressant like opioids because I am in pain all the time. Mentally and physically.,I hate people that abuse drugs and give them a bad name. Because I do not feel anything unless I take a stimulant or depressant like opioids because I am in pain all the time. Mentally and physically.
80,"This video was made when mephedrone first came to market and very little was known about the drug. Indeed, many people using it in the UK thought it was a plant extract of some sort. The video aimed to show it was in the same family of synthetic drugs such as crystal meth etc. Obviously, the precise nature of each of these drugs varies, but it is certainly true that drugs in this family offered the closest guide at the time as to what might be expected in terms of activity and side effects.",0,"This video was made when mephedrone first came to market and very little was known about the drug. Indeed, many people using it in the UK thought it was a plant extract of some sort. The video aimed to show it was in the same family of synthetic drugs such as crystal meth etc. Obviously, the precise nature of each of these drugs varies, but it is certainly true that drugs in this family offered the closest guide at the time as to what might be expected in terms of activity and side effects.","This video was made when mephedrone first came to market and very little was known about the drug. Indeed, many people using it in the UK thought it was a plant extract of some sort. The video aimed to show it was in the same family of synthetic drugs such as crystal meth etc. Obviously, the precise nature of each of these drugs varies, but it is certainly true that drugs in this family offered the closest guide at the time as to what might be expected in terms of activity and side effects."
94,"Mephedrone is whats in the good bathsalts, the bad shit has MDPV or Methylenedioxypyrovalerone....I'll stick with MDMA.",0,"Mephedrone is what is in the good bathsalts, the bad shit has MDPV or Methylenedioxypyrovalerone....I will stick with MDMA.","Mephedrone is what is in the good bathsalts, the bad shit has MDPV or Methylenedioxypyrovalerone....I will stick with MDMA."


In [40]:
# Save DF into CSV file
# df.to_csv("old_town_road_clean.csv")

In [41]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_bigram(spam_dropped['processed_text'], 21)
common_words = common_words[1:]

for word, freq in common_words:
    print(word, freq)


long term 8
does mean 5
term effects 5
legal highs 4
illegal drugs 4
addictive drug 4
plant food 4
meow meow 4
withdrawal symptoms 4
people say 4
stay away 4
drugs illegal 3
reason people 3
people just 3
drugs bad 3
know talking 3
like speed 3
does make 3
ketone group 3
chemical structure 3


In [42]:
df4 = pd.DataFrame(common_words, columns = ['CommentText' , 'count'])
df4.groupby('CommentText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in Comments after removing spams')

In [43]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

common_words = get_top_n_trigram(spam_dropped['processed_text'], 20)

for word, freq in common_words:
    print(word, freq)



long term effects 4
physical withdrawal symptoms 3
called plant food 2
cross blood brain 2
blood brain barrier 2
destroy body quickly 2
die short order 2
does physical withdrawal 2
best self incriminating 2
cutting cold turkey 2
people seen overdose 2
feels like cocaine 2
crystal meth does 2
reason taking drug 2
similar crystal meth 2
took mphedrone heavily 2
mphedrone heavily months 2
heavily months absolutely 2
months absolutely bad 2
absolutely bad effects 2


In [44]:
df6 = pd.DataFrame(common_words, columns = ['CommentText' , 'count'])
df6.groupby('CommentText').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar',
    yTitle='Count',
    linecolor='black',
    title='The 20 most frequent tri-grams in the dataset (without stopwords and spams)')

In [45]:
# Initialize Lemmatize
wordnet_lem = WordNetLemmatizer()

# Lemmatize processed text and join everything in a list
spam_dropped['text_lem'] = spam_dropped['processed_text'].apply(wordnet_lem.lemmatize)
all_words_lem = ' '.join([word for word in df['text_lem']])

KeyError: 'text_lem'

In [ ]:
# Generate a word cloud image
mask = np.array(Image.open("youtube.png"))
stopwords = set(STOPWORDS)

wordcloud_twitter = WordCloud(height=708,
                              width=1024,
                              background_color="white",
                              mode="RGBA",
                              stopwords=stopwords,
                              mask=mask).generate(all_words_lem)

# Create coloring from the image
image_colors = ImageColorGenerator(mask)
plt.figure(figsize=[20,20])
plt.axis('off')
plt.tight_layout(pad=0)
plt.imshow(wordcloud_twitter.recolor(color_func=image_colors), interpolation="bilinear")

# Store visualization to file
plt.savefig("yt_logo_unigram.png", format="png")

plt.show()

In [ ]:
word_to_remove = "lil nas"
new_string = ""

for word in spam_dropped['processed_text']:
    if word != word_to_remove:
        new_string += word + " "

new_string = new_string.strip()

In [ ]:
# Generate a word cloud image
mask = np.array(Image.open("youtube.png"))
stopwords = set(STOPWORDS)

wordcloud_twitter = WordCloud(height=2000,
                              width=2000,
                              background_color="white",
                              mode="RGBA",
                              stopwords=stopwords,
                              collocations = False,
                              mask=mask).generate(new_string)

# Create coloring from the image
image_colors = ImageColorGenerator(mask)
plt.figure(figsize=[20,20])
plt.axis('off')
plt.tight_layout(pad=0)
plt.imshow(wordcloud_twitter.recolor(color_func=image_colors), interpolation="bilinear")

# Store visualization to file
# plt.savefig("twitter_logo_mentions_wo_lilnas.png", format="png")

plt.show()